In [1]:
import numpy as np
import time

# importing Qiskit
from qiskit import Aer, IBMQ
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import register, execute

# import backend utilities
from qiskit.backends import JobStatus

# import tomography library
import qiskit.tools.qcvv.tomography as tomo

# useful additional packages 
from qiskit.tools.visualization import plot_state
from qiskit.tools.qi.qi import *
from qiskit.wrapper.jupyter import *
from qiskit.backends.ibmq import least_busy

from parseRealization import *
%matplotlib inline
global testDir
testDir="./tests/"

In [2]:
# Load saved IBMQ accounts
IBMQ.load_accounts()

KeyboardInterrupt: 

In [ ]:
def flipTheString(s):
    myString=""
    length = len(s)
    for it in range(0,length):
        myString=myString+s[length-1-it]
    return myString

def testFromFile(fileName,details=0):
    fileName1 = testDir+fileName + ".real"
    fileName2 = testDir+fileName + ".pla"
    size = readSizeFromFile(fileName1)
    answers = readKmapFromFile(fileName2)
    errors=[]
    error_count = 0
    backend_sim = Aer.get_backend('qasm_simulator')
    for i in answers:
        qr = QuantumRegister(size)
        cr = ClassicalRegister(size)
        qc = QuantumCircuit(qr,cr)

        for j in range(0,size):
            if i[j]=="1":
                qc.x(qr[j])
        qc,qr = readGatesFromFile(fileName1,qc,qr)
        qc.measure(qr,cr)
        job_sim = execute(qc, backend_sim)#,config)
        stats_sim = job_sim.result().get_counts()
        if details==1:
            # circuit_drawer(qc) won't work, why though
            print(i,stats_sim)
        for release in stats_sim:
            myString = flipTheString(release)
            if myString != answers[i]:
                error = i + ":" + myString + ":"+answers[i]
                errors = errors + [error]
                error_count = error_count+1
    if error_count != 0 :
        print("FAILURE")
    if details==1:
        print(qc.qasm())
        #quasmToMatlab(qc.qasm())
        print("Input:Received:Expected")
        for item in errors:
            print(item)
        print(answers)
        print("ERROR:",error_count)
    return error_count
            #plot_histogram(stats_sim)

In [ ]:
def measureFidelityOnSimulator(fileName):
    fileName1 = testDir+fileName + ".real"
    fileName2 = testDir+fileName + ".pla"
    size = readSizeFromFile(fileName1)
    answers = readKmapFromFile(fileName2)
    Unitary = getMatrixFromKmap( answers,size)
    print(Unitary)
    # compute Choi-matrix from unitary
    uni_choi = outer(vectorize(Unitary))
    t = []
    for i in range (0,size):
        t.append(i)
    #print(t)
    uni_tomo_set = tomo.process_tomography_set(t)
    print("Finished constructing tomography sets")
    qr = QuantumRegister(size)
    cr = ClassicalRegister(size)
    qc = QuantumCircuit(qr,cr, name=fileName)
    qc,qr = readGatesFromFile(fileName1,qc,qr)
    #print(qc.qasm())
    # Generate process tomography circuits
    uni_tomo_circuits = tomo.create_tomography_circuits(qc, qr, cr, uni_tomo_set)
    #for circuit in uni_tomo_circuits:
    #    print(circuit.name)
    max_credits = 8
    print("Finished constructing tomography circuits")

    backend = least_busy(IBMQ.backends(operational=True, simulator=False))
    shots = 100
    batch_size = int( pow(4,size)*pow(3,size) / 8)
    print(batch_size)
    num_batches = 8
    uni_tomo_jobs = []
    for j in range(num_batches):
        batch_circs = uni_tomo_circuits[j*batch_size:(j+1)*batch_size]
        uni_tomo_jobs.append(execute(batch_circs, backend=backend, shots=shots, max_credits=max_credits))
    
    
    uni_tomo_results = uni_tomo_jobs[0].result()
    print("Batch processed")
    for j in range(1,num_batches):
        uni_tomo_results += uni_tomo_jobs[j].result()
        print("Batch processed")
    
    

    print("Obtained Job results")
    uni_process_data = tomo.tomography_data( uni_tomo_results, fileName, uni_tomo_set)
    newSize = pow(2,size)
    uni_choi_fit = tomo.fit_tomography_data(uni_process_data, options={'trace':newSize})
    print('Process Fidelity = ', state_fidelity(vectorize(Unitary)/size, uni_choi_fit/newSize))
    #plot_state( uni_choi_fit)
    return 0

In [ ]:
fileName="swapcnotswapthreeQubits"
#fileName ="simpleSwap"
if (testFromFile(fileName)==0):
    measureFidelityOnSimulator(fileName)
